In [0]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys
import configparser
import logging
import inspect
from pyspark.sql.functions import count, lit, current_timestamp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType

# Configuración del logger
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger("py4j")
logger.setLevel(logging.WARN)
logger = logging.getLogger(__name__)

prod = True  # Para setear paths

# Variables globales

root_repo = "/Workspace/Shared/MITAFO"
config_files = {
    "general": f"{root_repo}/CGRLS_0010/Conf/CF_GRLS_PROCESS.py.properties",
    "connection": f"{root_repo}/CGRLS_0010/Conf/CF_GRLS_CONN.py.properties",
    "process": f"{root_repo}/ANCIN_0030/Jobs/04_GEN_ACRED_MOVS/DISPERSIONES/Conf/CF_PART_PROC.py.properties"
    if prod
    else "/Workspace/Repos/mronboye@emeal.nttdata.com/QueryConfigLab.ide/"
    "MITAFO/ANCIN_0030/Jobs/04_GEN_ACRED_MOVS/"
    "DISPERSIONES/Conf/"
    "CF_PART_PROC.py.properties",
}

notebook_name = (
    dbutils.notebook.entry_point.getDbutils()
    .notebook()
    .getContext()
    .notebookPath()
    .get()
)
message = "NB Error: " + notebook_name
source = "ETL"

process_name = "root"

# Carga de funciones externas
sys.path.append(f"{root_repo}/CGRLS_0010/Notebooks")
try:
    from NB_GRLS_DML_FUNCTIONS import *
    from NB_GRLS_SIMPLE_FUNCTIONS import *
except Exception as e:
    logger.error("Error al cargar funciones externas: %s", e)

global_params = {}
global_confs = {}  # Diccionario para almacenar las keys globales


def input_values() -> dict:
    """Obtiene los valores de los widgets de entrada y los almacena en un diccionario global."""

    widget_defaults = {
        "SR_FOLIO_REL": "",
        "SR_PROCESO": "",
        "SR_FECHA_LIQ": "",
        "SR_TIPO_MOV": "",
        "SR_REPROCESO": "",
        "SR_SUBPROCESO": "",
        "SR_USUARIO": "",
        "SR_INSTANCIA_PROCESO": "",
        "SR_ORIGEN_ARC": "",
        "SR_ID_SNAPSHOT": "",
        "SR_FECHA_ACC": "",
        "SR_FOLIO": "",
        "SR_SUBETAPA": "",
        "SR_FACTOR": "",
        "SR_ETAPA": "",
        "CX_CRE_ESQUEMA": "CIERREN_ETL",
        "TL_CRE_DISPERSION": "TTSISGRAL_ETL_DISPERSION",
    }

    # Crear los widgets en minúsculas
    for key, default_value in widget_defaults.items():
        dbutils.widgets.text(key.lower(), default_value)

    # Actualizar el diccionario global en mayúsculas para el resto del notebook
    global_params.update(
        {key.upper(): dbutils.widgets.get(key.lower()).strip() for key in widget_defaults}
    )

    if any(not value for value in global_params.values()):
        logger.error("Valores de entrada vacíos o nulos")
        global_params["status"] = "0"
    else:
        global_params["status"] = "1"

    return global_params


def conf_process_values(arg_config_file: str, arg_process_name: str) -> tuple:
    """Obtiene los valores de configuración del proceso y los almacena en un diccionario global."""
    keys = [
        "sql_conf_file",
        "debug",
        "conn_schema_001",
        "conn_schema_002",
        "table_001",
        "table_002",
        "table_003",
        "table_004",
        "table_005",
        "table_006",
        "table_007",
        "table_008",
        "table_009",
        "table_010",
        "table_011",
        "table_012",
        "table_013",
        "external_location",
        "err_repo_path",
        "output_file_name_001",
        "sep",
        "header",
        "catalog_name",
        "schema_name",
    ]

    try:
        config = configparser.ConfigParser()
        config.read(arg_config_file)
        result = {key: config.get(arg_process_name, key) for key in keys}
        result["status"] = "1"
        # Almacenar los valores en el diccionario global
        global_confs.update(result)
    except (ValueError, IOError) as error:
        logger.error("Error en la función %s: %s", inspect.stack()[0][3], error)
        result = {key: "0" for key in keys}
        result["status"] = "0"
        # Almacenar los valores en el diccionario global
        global_confs.update(result)

    return tuple(result.values())


def fix_created_file(file_name):
    try:
        file_name_tmp = dbutils.fs.ls(file_name + "_TEMP")
        file_name_new = list(filter(lambda x: x[0].endswith("csv"), file_name_tmp))
        dbutils.fs.mv(file_name_new[0][0], file_name)
        dbutils.fs.rm(file_name + "_TEMP", recurse=True)
    except Exception as e:
        logger.error("Function: " + str(inspect.stack()[0][3]))
        logger.error("An error was raised: " + str(e))
        return "0"
    return "1"

# Configuración del manejador global de excepciones
def global_exception_handler(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        # Permitir que KeyboardInterrupt se maneje normalmente
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    message = f"Uncaught exception: {exc_value}"
    source = "ETL"
    input_parameters = dbutils.widgets.getAll().items()
    # Registro del error y notificación
    logger.error("Please review log messages")
    notification_raised(webhook_url, -1, message, source, input_parameters)
    raise Exception("An error raised")

# Asigna el manejador de excepciones al hook global de sys
sys.excepthook = global_exception_handler


if __name__ == "__main__":
    # Inicialización de variables
    input_parameters = dbutils.widgets.getAll().items()

    webhook_url, channel, failed_task = conf_init_values(
        config_files["general"], process_name, "TEMP_PROCESS"
    )

    input_values()
    if global_params["status"] == "0":
        logger.error("Revisar mensajes en los logs")
        notification_raised(webhook_url, -1, message, source, input_parameters)
        raise Exception("Error en los valores de entrada, revisar logs")

    if failed_task == "0":
        logger.error("Please review log MESSAGEs")
        notification_raised(webhook_url, -1, message, source, input_parameters)
        raise Exception("Process ends")

    process_name = "root"
    conf_values = conf_process_values(config_files["process"], process_name)
    if conf_values[-1] == "0":
        logger.error("Revisar mensajes en los logs")
        notification_raised(webhook_url, -1, message, source, input_parameters)
        raise Exception("Error en la configuración del proceso, revisar logs")

    conn_name_ora = "jdbc_oracle"
    (
        conn_options,
        conn_additional_options,
        conn_user,
        conn_key,
        conn_url,
        scope,
        failed_task,
    ) = conf_conn_values(config_files["connection"], conn_name_ora)
    if failed_task == "0":
        logger.error("Revisar mensajes en los logs")
        notification_raised(webhook_url, -1, message, source, input_parameters)
        raise Exception("Error en la configuración de la conexión, revisar logs")

    if prod:
        sql_conf_file = f"{root_repo}/ANCIN_0030/Jobs/04_GEN_ACRED_MOVS/DISPERSIONES/JSON/{conf_values[0]}"
    else:
        sql_conf_file = f"/Workspace/Repos/mronboye@emeal.nttdata.com/QueryConfigLab.ide/MITAFO/ANCIN_0030/Jobs/04_GEN_ACRED_MOVS/DISPERSIONES/JSON/{conf_values[0]}"
    # Seteamos el valor de debug
    debug = conf_values[1]
    debug = debug.lower() == 'true'

In [0]:
with open(sql_conf_file) as f:
    file_config_sql = json.load(f)

conf_values = [
    (fields["step_id"], "\n".join(fields["value"]))
    for line, value in file_config_sql.items()
    if line == "steps"
    for fields in value
]

### Hacemos un query que tiene multiples extracciones y transformaciones en OCI

In [0]:
query_statement = "011"
table_name_001 = f"{global_confs['conn_schema_001']}.{global_confs['table_002']}" # CIERREN_ETL.TTSISGRAL_ETL_MOVIMIENTOS
table_name_002 = f"{global_confs['conn_schema_001']}.{global_confs['table_012']}" # CIERREN_ETL.TTSISGRAL_ETL_VAL_IDENT_CTE
table_name_003 = f"{global_confs['conn_schema_002']}.{global_confs['table_009']}" # CIERREN_ETL.TCAFOGRAL_VALOR_ACCION
table_name_004 = f"{global_confs['conn_schema_002']}.{global_confs['table_013']}" # CIERREN_ETL.TCCRXGRAL_CAT_CATALOGO

params = [
    table_name_001,
    table_name_002,
    table_name_003,
    table_name_004,
    global_params["SR_FOLIO"],
]

statement, failed_task = getting_statement(conf_values, query_statement, params)

if failed_task == "0":
    logger.error("No value %s found", statement)
    logger.error("Please review log messages")
    notification_raised(webhook_url, -1, message, source, global_params)
    raise Exception("Process ends")

# Ensure the statement is correctly formatted with the parameters
formatted_statement = statement.format(*params)

df, failed_task = query_table(
    conn_name_ora, spark, formatted_statement, conn_options, conn_user, conn_key
)

# Inserto DF_100_CRE_ETL_DISPERSION al cache
df.cache()

if failed_task == "0":
    logger.error("Please review log messages")
    notification_raised(webhook_url, -1, message, source, input_parameters)
    raise Exception("An error raised")

if debug:
    display(df)

### Generamos un .csv

In [0]:
from datetime import datetime
import pytz

header = True

full_file_name = (
    global_confs["external_location"]
    + global_confs["err_repo_path"]
    + "/"
    + global_params["SR_FOLIO"]
    + "_"
    + global_confs["output_file_name_001"]
)

try:
    # Guarda temporalmente el DataFrame en formato CSV
    df.write.format("csv").mode("overwrite").option("header", header).save(
        full_file_name + "_TEMP"
    )

    # Lee el archivo temporal en un nuevo DataFrame
    dataframe = spark.read.option("header", header).csv(full_file_name + "_TEMP")

    # Guarda el DataFrame en un solo archivo CSV
    dataframe.coalesce(1).write.format("csv").mode("overwrite").option(
        "header", header
    ).save(full_file_name)
except Exception as e:
    # Registra el error en el logger
    logger.error("An error was raised: " + str(e))
    # Envía una notificación de error
    notification_raised(webhook_url, -1, message, source, input_parameters)
    # Lanza una excepción para finalizar el proceso
    raise Exception("Process ends")

In [0]:
try:
    # Lee el archivo CSV en un DataFrame, con la opción de encabezado según la variable 'header'
    dfFile = spark.read.option('header', header).csv(full_file_name)
    
    # Escribe el DataFrame en un solo archivo CSV, con la opción de encabezado según la variable 'header'
    dfFile.coalesce(1).write.format('csv').mode('overwrite').option('header', header).save(full_file_name)
except Exception as e:
    # Registra el error en el logger
    logger.error("An error was raised: " + str(e))
    
    # Envía una notificación de error
    notification_raised(webhook_url, -1, message, source, input_parameters)
    
    # Lanza una excepción para finalizar el proceso
    raise Exception("Process ends")

In [0]:
try:
    # Define el nombre del archivo temporal auxiliar
    full_file_name_aux = full_file_name + '_TEMP'
    
    # Elimina el archivo temporal auxiliar si existe
    dbutils.fs.rm(full_file_name_aux, recurse=True)

    # Lista los archivos en el directorio del archivo original
    file_name_tmp = dbutils.fs.ls(full_file_name)
    
    # Filtra la lista de archivos para encontrar el archivo CSV
    file_name_new = list(filter(lambda x: x[0].endswith('csv'), file_name_tmp))

    # Copia el archivo CSV encontrado al archivo temporal auxiliar
    dbutils.fs.cp(file_name_new[0][0], full_file_name_aux)
    
    # Elimina el archivo original
    dbutils.fs.rm(full_file_name, recurse=True)
    
    # Copia el archivo temporal auxiliar al nombre del archivo original
    dbutils.fs.cp(full_file_name_aux, full_file_name)
    
    # Elimina el archivo temporal auxiliar
    dbutils.fs.rm(full_file_name_aux, recurse=True)
except Exception as e:
    # Registra el error en el logger
    logger.error("An error was raised: " + str(e))
    
    # Envía una notificación de error
    notification_raised(webhook_url, -1, message, source, input_parameters)
    
    # Lanza una excepción para finalizar el proceso
    raise Exception("Process ends")

In [0]:
# # Copia el archivo generado al volumen asignado para poder descargarlo en el equipo local
# dbutils.fs.cp(
#    f"abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/RCDI/OUT/{global_params['SR_FOLIO']}_INSUFICIENCIA.csv", 
#    f"/Volumes/dbx_mit_dev_1udbvf_workspace/default/doimss_carga_archivo/{global_params['SR_FOLIO']}_INSUFICIENCIA.csv"
# )

In [0]:
from pyspark.sql import DataFrame

# Clear cache
spark.catalog.clearCache()

# Unpersist and delete all DataFrames
for df_name in list(globals()):
    if isinstance(globals()[df_name], DataFrame):
        globals()[df_name].unpersist()
        del globals()[df_name]

In [0]:
# Liberar la caché del DataFrame si se usó cache
df.unpersist()

# Eliminar DataFrames para liberar memoria
del df

# Recolector de basura para liberar recursos inmediatamente
import gc
gc.collect()